In [9]:
import sys
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')
#warnings.filterwarnings(action='once')

## Final Tf-idf based model

In [10]:
#process=True
process=False

if process:
    df=pd.read_csv("processed_description_sine_error.csv")

    df.shape

    df=df[df["processed_description"].isna()!=True]

    bad_words=["www", "http", "https", "th", "pm", "ticket", "org", "event", "link", "registr",
               "meetup", "event", "group", "regist", "pleas", "please", "join", "rsvp", "member", 
               "venu", "free", "comment", "thank", "attend", "eventbrit", "mr", "st", "rd", "hour", "new", "time", 
               "boston", "like"   ]
    def clean_description(desc):
        good_words=[elem for elem in desc.split() if elem not in bad_words]
        return ' '.join(good_words)


    """desc_try=df['description'][0]
    desc_try

    clean_description(desc_try)""";

    from multiprocessing import cpu_count, Pool

    #cores = cpu_count() #Number of CPU cores on your system
    cores=4
    partitions = cores #Define as many partitions as you want

    def parallelize(data, func):
        data_split = np.array_split(data, partitions)
        pool = Pool(cores)
        data = pd.concat(pool.map(func, data_split))
        pool.close()
        pool.join()
        return data
    #df["yolo"]="NaN"
    def test_func(data):
        #print "Process working on: ",data
        data['processed_description'] = data.processed_description.map(clean_description)
        return data

    df = parallelize(df, test_func)

    df.head()

    #df.to_csv("processed_df_no_badwords.csv")
else:
    df=pd.read_csv("processed_df_no_badwords.csv")

In [11]:
df.head()

,Unnamed: 0,description,category_bin,lang_ok,processed_description
0,0,<p>FOOD AND FELLOWSHIP | 5 PM</p> \n<p>CONCERT...,27,english,food fellowship concert may us morn star churc...
1,1,<p><b>【WhyNot!?JAPAN + MeetUp Collaboration Ev...,28,english,whynot japan collabor peopl come whynot japan ...
2,2,<p>TENTATIVE<br></p>\n<p><br></p>\n<p>10.30 - ...,31,english,tentat introduct orient session break network ...
3,3,"<p><span>Hey explorersssss, what's up?</span><...",15,english,hey explorersssss say night view hong kong vic...
4,4,"<p>Free, unauditioned, collaborative pop choir...",18,english,unaudit collabor pop choir identifi women see ...


In [12]:
df=df[df["processed_description"].isna()!=True]

In [13]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import classification_report, roc_curve, auc

In [14]:
X_data = df['processed_description']
Y_data = df['category_bin']

X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size=.25)

In [15]:
tf_vectorizer = TfidfVectorizer(analyzer = "word", 
                                tokenizer = None, 
                                norm = 'l1', 
                                preprocessor = None, 
                                max_features = 10000)

In [16]:
#load=True
load=False

if load:
    import joblib
    X_train_tfidf=joblib.load( "try_retreat_X_train_tfidf.joblib")
    X_test_vec_tf=joblib.load( "try_retreat_X_test_vec_tf.joblib")
else:
    X_train_tfidf = tf_vectorizer.fit_transform(X_train)
    #X_train_tfidf = X_train_tfidf.toarray()
    X_train_tfidf.shape
    X_test_vec_tf = tf_vectorizer.transform(X_test)
    X_test_vec_tf = X_test_vec_tf.toarray()

In [23]:
with open('vectorizer.pickle', 'wb') as handle:
    pickle.dump(tf_vectorizer, handle)

In [17]:
save=False

if save:
    import joblib
    joblib.dump(X_train_tfidf, "try_retreat_X_train_tfidf.joblib")

    joblib.dump(X_test_vec_tf, "try_retreat_X_test_vec_tf.joblib")

    

In [18]:
#del X_train_tfidf
del df

In [21]:
forest = RandomForestClassifier(n_jobs=5, n_estimators=20)#, n_estimators=160)#, n_estimators=10, bootstrap=False, random_state=31456 ) 
forest = forest.fit(X_train_tfidf, Y_train)

In [22]:
forest.score(X_test_vec_tf, Y_test)#10 est

0.7416269113149847

In [ ]:
predictions = forest.predict(X_test_vec_tf)

In [ ]:
print(metrics.classification_report(Y_test, predictions)) #200 features

In [22]:
forest.score(X_test_vec_tf, Y_test)#20 est

0.7377422196124486

In [26]:
forest.score(X_test_vec_tf, Y_test)#50 est

0.7484096692111959

In [25]:
forest.score(X_test_vec_tf, Y_test)#100 est

0.7523732628694461

In [32]:
forest.score(X_test_vec_tf, Y_test)#150 est

0.7517371305539244

In [ ]:
forest.score(X_test_vec_tf, Y_test)#200 est

In [ ]:
predictions = forest.predict(X_test_vec_tf)


In [ ]:
print(metrics.classification_report(Y_test, predictions)) #200 features

In [31]:
print(metrics.classification_report(Y_test, predictions)) #150 features

              precision    recall  f1-score   support

           0       0.93      0.51      0.66       403
           1       0.90      0.78      0.84       180
           2       0.68      0.59      0.63       305
           3       0.77      0.84      0.80      3912
           4       0.91      0.59      0.72       298
           5       0.90      0.48      0.62       733
           6       0.82      0.84      0.83       927
           7       0.83      0.53      0.65       820
           8       0.91      0.35      0.50        84
           9       0.81      0.63      0.71       884
          10       0.82      0.75      0.79      1927
          11       0.80      0.60      0.68      1231
          12       0.82      0.86      0.84      1236
          13       0.72      0.81      0.76      3801
          14       0.86      0.59      0.70       392
          15       0.84      0.70      0.76      1473
          16       0.90      0.74      0.82       654
          17       0.76    

In [19]:
print(metrics.classification_report(Y_test, predictions)) #100 features

              precision    recall  f1-score   support

           0       0.94      0.51      0.66       403
           1       0.90      0.78      0.84       180
           2       0.71      0.58      0.64       305
           3       0.76      0.84      0.80      3912
           4       0.92      0.58      0.71       298
           5       0.89      0.47      0.61       733
           6       0.82      0.84      0.83       927
           7       0.83      0.53      0.65       820
           8       0.91      0.35      0.50        84
           9       0.79      0.64      0.71       884
          10       0.84      0.75      0.79      1927
          11       0.78      0.61      0.68      1231
          12       0.82      0.86      0.84      1236
          13       0.72      0.81      0.77      3801
          14       0.87      0.60      0.71       392
          15       0.82      0.70      0.76      1473
          16       0.89      0.74      0.81       654
          17       0.75    

In [33]:
print(metrics.classification_report(Y_test, predictions)) #100 features

              precision    recall  f1-score   support

           0       0.96      0.49      0.65       401
           1       0.94      0.81      0.87       167
           2       0.72      0.54      0.62       338
           3       0.77      0.84      0.80      3815
           4       0.91      0.61      0.73       328
           5       0.87      0.50      0.63       689
           6       0.80      0.87      0.83      1007
           7       0.84      0.55      0.67       802
           8       0.77      0.36      0.49        84
           9       0.82      0.64      0.72       884
          10       0.84      0.78      0.81      1888
          11       0.79      0.60      0.68      1156
          12       0.79      0.86      0.82      1257
          13       0.73      0.81      0.77      3790
          14       0.81      0.60      0.69       401
          15       0.83      0.70      0.76      1474
          16       0.89      0.77      0.82       658
          17       0.75    

In [28]:
print(metrics.classification_report(Y_test, predictions)) #50 features

              precision    recall  f1-score   support

           0       0.96      0.48      0.64       401
           1       0.93      0.81      0.87       167
           2       0.72      0.53      0.61       338
           3       0.75      0.83      0.79      3815
           4       0.91      0.62      0.74       328
           5       0.84      0.50      0.63       689
           6       0.79      0.86      0.82      1007
           7       0.82      0.55      0.66       802
           8       0.77      0.36      0.49        84
           9       0.80      0.64      0.71       884
          10       0.83      0.77      0.80      1888
          11       0.77      0.61      0.68      1156
          12       0.78      0.86      0.82      1257
          13       0.73      0.80      0.76      3790
          14       0.78      0.59      0.67       401
          15       0.84      0.69      0.75      1474
          16       0.88      0.77      0.82       658
          17       0.73    

In [24]:
print(metrics.classification_report(Y_test, predictions)) #20 features

              precision    recall  f1-score   support

           0       0.92      0.48      0.64       401
           1       0.94      0.81      0.87       167
           2       0.68      0.53      0.59       338
           3       0.74      0.83      0.78      3815
           4       0.85      0.63      0.72       328
           5       0.78      0.51      0.61       689
           6       0.78      0.85      0.81      1007
           7       0.77      0.55      0.64       802
           8       0.79      0.32      0.46        84
           9       0.76      0.63      0.69       884
          10       0.81      0.77      0.79      1888
          11       0.73      0.61      0.67      1156
          12       0.76      0.84      0.80      1257
          13       0.71      0.79      0.75      3790
          14       0.79      0.60      0.68       401
          15       0.78      0.69      0.73      1474
          16       0.87      0.75      0.80       658
          17       0.70    

In [15]:
print(metrics.classification_report(Y_test, predictions)) #10 est

              precision    recall  f1-score   support

           0       0.74      0.50      0.60       401
           1       0.92      0.81      0.86       167
           2       0.57      0.50      0.54       338
           3       0.69      0.83      0.75      3815
           4       0.81      0.63      0.71       328
           5       0.68      0.51      0.58       689
           6       0.74      0.84      0.79      1007
           7       0.70      0.54      0.61       802
           8       0.74      0.35      0.47        84
           9       0.71      0.64      0.67       884
          10       0.76      0.78      0.77      1888
          11       0.66      0.61      0.63      1156
          12       0.74      0.82      0.78      1257
          13       0.70      0.77      0.74      3790
          14       0.76      0.57      0.65       401
          15       0.77      0.68      0.72      1474
          16       0.88      0.74      0.80       658
          17       0.69    

In [13]:
forest_100_est=forest

In [4]:
import joblib
#joblib.dump(forest_100_est,"final_tfidf_model.joblib")

In [5]:
a=joblib.load("final_tfidf_model.joblib")

In [6]:
a

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=5,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [7]:
import pickle

In [8]:
with open('model.pickle', 'wb') as handle:
    pickle.dump(a, handle)